In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [34]:
def get_data(filename):
  file = np.genfromtxt(filename, dtype='int32', delimiter=',')
  row_mask = (file != -1).all(axis=1) #usunięcie próbek z wartościami -1 (brak wartości pewnej cechy w datasecie)
  file = file[row_mask, :]
  return file

In [35]:
def compute_chi2_stats(X, y):
  chi2vals, pvals = chi2(X, y)
  return chi2vals, pvals

In [36]:
def select_k_best_via_chi2(k, X, y):
  X = SelectKBest(chi2, k).fit_transform(X, y)
  return X

In [37]:
def do_cross_validation(X, y, state): #do zmiany
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=state)
  return X_train, X_test, y_train, y_test

In [38]:
def LCG(arg): # linear congruential generator
  return (187668917 * arg + 11767183) % 2147483647
def get_next_random(arg=0, depth=0):
  if depth == 0:
    return LCG(arg)
  return get_next_random(LCG(arg), depth - 1)

In [39]:
def do_nth_cross_validation(X, y, n):
  X_train, X_test, y_train, y_test = do_cross_validation(X, y, get_next_random(42, n))
  return X_train, X_test, y_train, y_test

In [40]:
class NaiveBayesClassifier():
  def __init__(self):
    pass

  # liczymy prawdopodobienstwa a priori klas
  def calculate_prior(self, y_train):
    self.labels, self.counts = np.unique(y_train, return_counts=True)
    probabilities = self.counts / np.sum(self.counts)
    self.prior_probs = dict(zip(self.labels, probabilities))

  # liczymy parametry rozkładów w klasach
  def calculate_gaussian_parameters(self, X_train, y_train):
    self.gauss_params = {}
    for label in self.labels:
      mean_std_for_features = []
      for feature in range(X_train.shape[1]):
        class_indices = np.where(y_train == label)[0]
        feature_values_for_class = X_train[class_indices, feature]
        mean_std_tuple = np.mean(feature_values_for_class), np.std(feature_values_for_class)
        mean_std_for_features.append(mean_std_tuple)
      self.gauss_params[label] = mean_std_for_features
  
  # dopasowanie modelu do danych treningowych
  def fit(self, X_train, y_train):
    self.calculate_prior(y_train)
    self.calculate_gaussian_parameters(X_train, y_train)

  def predict(self, X_test):
    self.predicted_labels = np.zeros(X_test.shape[0])
    for sample in X_train:
      prediction = self.labels[0]
      MAP_value = -1
      for label in self.labels:
        Bayes_product = prior_probs[label]
        for i in range(len(feature)):
          Bayes_product *= 



SyntaxError: ignored

In [ ]:
data = get_data("/content/drive/My Drive/Colab Notebooks/breast-cancer-wisconsin.txt")
X = data[:, 1:10] # macierz cech, bez id i klas
y = data[:, 10]   # wektor klas
chi2val, pval = compute_chi2_stats(X, y) 
print(chi2val)
X_new  = select_k_best_via_chi2(5, X, y)
X_train, X_test, y_train, y_test = do_nth_cross_validation(X_new, y, 5)

NBC = NaiveBayesClassifier()
NBC.fit(X_train, y_train)
print(NBC.gaussian_parameters_for_classes)

In [ ]:
for i in range(5):
  X_train, X_test, y_train, y_test = do_nth_cross_validation(X, y, i)